## Understanding the dataset:

This dataset was from a [Kaggle Competition](https://www.kaggle.com/competitions/brist1d) that ran in Fall 2024. It describes blood glucose levels, measured in mmol/L, over the past 6 hours in a tabular format with 5 minute increments. Additionally, the dataset includes information on insulin doses received, carbohydrates consumed, mean heart rate, steps taken, calories burnt, and self-declared activity performed. 

The target variable was to predict blood glucose levels 1 hour into the future, given this past data from the past 6 hours. 

Data cleaning was necessary due to the large number of missing data points. Feature engineering was also used to better reflect a continuous time series with both micro and macro trends.

## Intro/data imports

Below is a full notebook detailing my submission for the BrisT1D Blood Glucose Prediction Competition, hosted by Kaggle. Datasets are available on [Kaggle](https://www.kaggle.com/competitions/brist1d/overview) - place all CSV files in a directory named 'brist1d' - left empty for demonstrative purposes. The following python packages are used in this notebook:
- Pandas
- Numpy
- Matplotlib
- Scikit-Learn
- XGBoost
- CatBoost
- Pytorch
- Weights & Biases (optional, used to tune hyperparameters)

In [158]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.model_selection import train_test_split, cross_val_score, GroupKFold
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import root_mean_squared_error

from xgboost import XGBRegressor
from catboost import CatBoostRegressor

RANDOM_SEED = 42

In [159]:
df_train = pd.read_csv('brist1d/train.csv',
                        index_col='id', 
                        parse_dates=['time'])

df_test = pd.read_csv('brist1d/test.csv', 
                       index_col='id', 
                       parse_dates=['time'])

df_subm = pd.read_csv('brist1d/sample_submission.csv',
                       index_col='id')

df_train.columns = df_train.columns.str.replace(':', '-')
df_test.columns = df_test.columns.str.replace(':', '-')

### Dataframe memory reduction:

In [160]:
def reduceMemory(df):
    df = df.copy()

    for col in df.columns:
        if pd.api.types.is_float_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce', downcast='float')
        elif pd.api.types.is_object_dtype(df[col]):
            df[col] = df[col].astype('category')

    return df

print(f"Memory usage of df_train before reduction: {df_train.memory_usage().sum() / (1024 * 1024):.2f} MB")

df_train = reduceMemory(df_train)
df_test = reduceMemory(df_test)

print(f"Memory usage of df_train after reduction: {df_train.memory_usage().sum() / (1024 * 1024):.2f} MB")

Memory usage of df_train before reduction: 686.10 MB
Memory usage of df_train after reduction: 307.48 MB


## Data cleaning and feature engineering

Due to the time-series lags in this dataset, NaNs are often propagated along multiple rows that are actually the same value. To ensure similar filling, I used row interpolation to fill the data based on nearby neighbors.

Additionally, time series lags exist up to 6 hours into the past for this dataset. I found that including more than 120 minutes of data did not improve model performance, so below also includes limiting columns to the latest 24 (5 min x 24 = 120 minutes).

In [161]:
columns_list = df_train.columns

bg_cols = [col for col in columns_list if col.startswith('bg-0') or col.startswith('bg-1')]
cals_cols = [col for col in columns_list if col.startswith('cals-0') or col.startswith('cals-1')]
carbs_cols = [col for col in columns_list if col.startswith('carbs-0') or col.startswith('carbs-1')]
insulin_cols = [col for col in columns_list if col.startswith('insulin-0') or col.startswith('insulin-1')]
hr_cols = [col for col in columns_list if col.startswith('hr-0') or col.startswith('hr-1')]
steps_cols = [col for col in columns_list if col.startswith('steps-0') or col.startswith('steps-1')]

for cols in [bg_cols, cals_cols, carbs_cols, insulin_cols, hr_cols, steps_cols]:
    df_train[cols] = df_train[cols].interpolate(axis=1, limit_direction='both')
    df_test[cols] = df_test[cols].interpolate(axis=1, limit_direction='both')

# Carbs, steps, and cals likely have NaNs meaning 0, not missing information
df_train[cals_cols] = df_train[cals_cols].fillna(0)
df_test[cals_cols] = df_test[cals_cols].fillna(0)

df_train[carbs_cols] = df_train[carbs_cols].fillna(0)
df_test[carbs_cols] = df_test[carbs_cols].fillna(0)

df_train[steps_cols] = df_train[steps_cols].fillna(0)
df_test[steps_cols] = df_test[steps_cols].fillna(0)

feature_cols = bg_cols + cals_cols + carbs_cols + insulin_cols + hr_cols + steps_cols

Finally impute remaining features based on means of their respective columns. I tried multiple more complicated imputing methods including splines and iterative imputing, but neither gave a performance improvement to the model. KNN imputation was too computationally expensive for the number of rows in the dataset (due to the nonlinear time complexity of searching for the closest neighbors/rows).

In [162]:
mean_imputer = SimpleImputer()

df_train[feature_cols] = mean_imputer.fit_transform(df_train[feature_cols])
df_test[feature_cols] = mean_imputer.transform(df_test[feature_cols])

### Feature engineering

For `carbs`, `steps`, and `cals`, I wanted to add running cumulative summations across the 2 hour window.

In [163]:
df_train[[f'cumsum_{col}' for col in cals_cols]] = df_train[[col for col in cals_cols]].cumsum(axis=1)
df_test[[f'cumsum_{col}' for col in cals_cols]] = df_test[[col for col in cals_cols]].cumsum(axis=1)

df_train[[f'cumsum_{col}' for col in carbs_cols]] = df_train[[col for col in carbs_cols]].cumsum(axis=1)
df_test[[f'cumsum_{col}' for col in carbs_cols]] = df_test[[col for col in carbs_cols]].cumsum(axis=1)

df_train[[f'cumsum_{col}' for col in steps_cols]] = df_train[[col for col in steps_cols]].cumsum(axis=1)
df_test[[f'cumsum_{col}' for col in steps_cols]] = df_test[[col for col in steps_cols]].cumsum(axis=1)

cumsum_cols = [col for col in df_train.columns if col.startswith('cumsum')]

feature_cols.extend(cumsum_cols)

Add sin and cos values for the hour (for example to ensure 11pm smoothly translates to 12am from the `time` column).

In [164]:
df_train['sin_hour'] = np.sin(np.pi * df_train['time'].dt.hour / 12)
df_train['cos_hour'] = np.cos(np.pi * df_train['time'].dt.hour / 12)

df_test['sin_hour'] = np.sin(np.pi * df_test['time'].dt.hour / 12)
df_test['cos_hour'] = np.cos(np.pi * df_test['time'].dt.hour / 12)

feature_cols.extend(['sin_hour', 'cos_hour'])

Add different windows of means and variances for blood glucose, to capture any significance in variance on predictability of future blood glucose levels.

In [165]:
for window in [3, 6, 9]:
    df_train[f'bg_mean_{window}'] = df_train[bg_cols[-window:]].mean(axis=1)
    df_train[f'bg_var_{window}'] = df_train[bg_cols[-window:]].var(axis=1)
    df_test[f'bg_mean_{window}'] = df_test[bg_cols[-window:]].mean(axis=1)
    df_test[f'bg_var_{window}'] = df_test[bg_cols[-window:]].var(axis=1)
    feature_cols.extend([f'bg_mean_{window}', f'bg_var_{window}'])

I attempted to include the column `activity` using a target encoder or similar. However, most activities appeared less than 20 times in the training dataset. Furthermore, of those who appeared more frequently in the training dataset, they did not show up enough in the test dataset to be worth including.

Final preparations include using a standard scaler and splitting the dataframes.

In [166]:
std_scaler = StandardScaler()

df_train[feature_cols] = std_scaler.fit_transform(df_train[feature_cols])
df_test[feature_cols] = std_scaler.transform(df_test[feature_cols])

X_train = df_train[feature_cols].copy()
y_train = df_train['bg+1-00'].copy()

X_test = df_test[feature_cols].copy()

stratify_groups = df_train['p_num'].copy()

## Model 1: Gradient Boosting Decision Trees in a Stacking Regressor

An easy start for regressions on tabular data are GBDTs - I used XGBoost and CatBoost as an ensemble for this dataset. For hyperparameter tuning, I used bayesian hyperparameter optimization, specifically employing [Weights & Biases](https://wandb.ai/site/) for the informative visualization and tracking of results. Out of sample validation was performed to reduce overfitting.

Below is the full parameter grid used. For purposes of keeping this notebook consolidated, below is the function used to tune the hyperparameters. Ensure you have `wandb` installed and the final line is uncommented if you choose to run the tuning yourself.

In [167]:
import wandb

sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'rmse', 'goal': 'minimize'},
    'parameters': {
        'xgb_n_estimators': {'min': 1000, 'max': 4000},
        'xgb_learning_rate': {'min': 0.005, 'max': 0.2},
        'xgb_max_depth': {'min': 4, 'max': 8},
        'catboost_iterations': {'min': 1000, 'max': 4000},
        'catboost_depth': {'min': 4, 'max': 8}
    }
}

def train_model():
    run = wandb.init()
    config = wandb.config

    xgb = XGBRegressor(
        n_estimators = config.xgb_n_estimators,
        learning_rate= config.xgb_learning_rate,
        max_depth = config.xgb_max_depth,
        random_state = RANDOM_SEED,
        eval_metric = 'rmse'
    )

    catboost = CatBoostRegressor(
        iterations = config.catboost_iterations,
        depth = config.catboost_depth,
        random_seed = RANDOM_SEED,
        verbose = False
    )

    estimators = [
        ('xgb', xgb),
        ('catboost', catboost)
    ]

    final_estimator = RidgeCV(alphas=[1e-2, 1e-1, 1])

    stacking_model = StackingRegressor(
        estimators = estimators, 
        final_estimator = final_estimator,
        n_jobs = -1
    )

    X_split, X_val, y_split, y_val = train_test_split(X_train, y_train)

    stacking_model.fit(X_split, y_split)
    y_pred = stacking_model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)

    wandb.log({'rmse': rmse,
               'ridge_alpha': stacking_model.final_estimator_.alpha_})

def main():
    sweep_id = wandb.sweep(sweep_config, project='kaggle-bg-gbdt-sweep')

    wandb.agent(sweep_id, function=train_model, count=50)

# # Uncomment to run:
# main()

Below is the visualization of best parameters after 13 runs. The best out-of-sample RMSE remained at around 1.50.

![img](wandb_chart.png)

The best hyperparemters for this stacking regressor are as follows:

- Catboost depth: 8
- Catboost iterations: 1800
- XGBoost learning rate: 0.15
- XGBoost max depth: 8
- XGBoost number of estimators: 2800
- Final estimator (RidgeCV) alpha: 1.0

Final parameters after decided by the Weights & Biases search are used in the models below to fit the full dataset. I additionally used a 5-fold cross validation on the best hyperparameter set to understand the range of RMSE scores (using +/- 2 times the standard deviation) 

The model with the tuned parameters is then used to predict the `bg+1:00` column in the test dataset, which is then ready for submission.

In [168]:
xgb_final = XGBRegressor(
    n_estimators = 2800,
    learning_rate = 0.15,
    max_depth = 8,
    random_state = RANDOM_SEED,
    eval_metric='rmse'
)

catboost_final = CatBoostRegressor(
    iterations = 1800,
    depth = 8,
    random_seed=RANDOM_SEED,
    verbose=False
)

estimators = [
    ('xgb', xgb_final),
    ('catboost', catboost_final)
]

final_estimator = Ridge(alpha=1.0)

stacking_model = StackingRegressor(
    estimators=estimators, 
    final_estimator=final_estimator,
    n_jobs=-1
)

cv_scores = cross_val_score(
    stacking_model,
    X_train,
    y_train,
    cv=GroupKFold(n_splits=5),
    groups=stratify_groups,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

print(f"CV RMSE: {-cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

stacking_model.fit(X_train, y_train)
y_pred_gbdt = stacking_model.predict(X_test)

CV RMSE: 2.1326 (+/- 0.2112)


## Model 2: Multilayer perceptron neural network with tuned regularization

This model was inspired by the analysis of the paper ["Well-tuned Simple Nets Excel on Tabular Datasets"](https://arxiv.org/abs/2106.11189) by Kadra et. all, which I came across when doing research on the effectiveness on TabNets, a popular technique for tabular learning.

The authors findings were simple - using a multilayer perceptron neural network with a "cocktail" of regularization features, these neural networks can perform better than GBDTs on many tabular datasets. I wanted to try to implement my own python class around some of these regularization techniques, in hopes to refine and add more regularization techniques in the future. The full model architecture is in `MLP_Regressor.py`, and is called in this file for tuning and use.

In [169]:
from MLP_Regressor import MLPRegressor

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

# convert to float32 for MPS torch support
X_split, X_val, y_split, y_val = train_test_split(X_train.astype(np.float32).values, y_train.astype(np.float32).values, test_size=0.2, random_state=RANDOM_SEED)

# I default to mps as I use an Apple silicon macbook
device = torch.device('mps' if torch.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')

X_split_tensor = torch.from_numpy(X_split).to(device)
y_split_tensor = torch.from_numpy(y_split).to(device)
X_val_tensor = torch.from_numpy(X_val).to(device)
y_val_tensor = torch.from_numpy(y_val).to(device)

train_dataset = TensorDataset(X_split_tensor, y_split_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=128)
val_loader = DataLoader(val_dataset, batch_size=128)

The below cell defines the training loop for this MLP Regressor. While I don't have the compute power to tune hyperparameters of a large neural network, a tool like [Weights & Biases](https://wandb.ai/site/) or [Optuna](https://optuna.org) use SOTA algorithms to efficiently find the best regularization cocktail. The below parameters provide reasonable RMSE results, though could be improved with hyperparameter optimization. 

Due to the option of snapshot ensembles, models are saved to the folder `best_models` - when loading these models, be sure to run inputs through each model and take the mean output to effectively leverage the ensemble. 

To go more in depth on the architectural choices for this model, the MLP Regressor class is meant to work closely with the below training loop. This is evident through optimizer and scheduler initialization done through the MLP class, as an AdamW optimizer and Cosine Annealing scheduler are paramount for weight decay and snapshot ensembling, respectively. I aimed to remove hyperparameter tuning from the training function to simplify parameter tuning (despite the need for snapshot ensembles or early stopping to be called during the training loop), so I created them as parameters in the model that are then called by the training loop.

`ensemble_load` and `ensemble_predict` in the MLP Regressor class are helper functions for the snapshot ensemble process - when the cosine annealer resets its cycle, I only save the `state_dict` of the final model in the cycle. `ensemble_load` is used to initialize models with the same parameters and state_dict as they were saved from. `ensemble_predict` uses these models to predict a given X input, helpful for validation testing of the OOS data during training.

In [170]:
### Model setup

from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
import os

input_dim = X_train.shape[1]
hidden_dims = [512, 512, 256, 128, 64]

model = MLPRegressor(
    input_dim=input_dim, 
    hidden_dims=hidden_dims,
    learning_rate=1e-2,
    use_snapshots=True,
    cycle_length=30,
    cycle_mult=1,
    batch_normalization=True,
    weight_decay=0.01,
    dropout_rate=0.4,
    early_stopping_patience=150,
    activation_fn=nn.ReLU()
)

model.to(device)

optimizer = model.get_optimizer()
scheduler = model.get_scheduler(optimizer)

# create a dir for saving the best model - useful for snapshot ensembles with multiple models present
save_dir = 'best_models'
os.makedirs(save_dir, exist_ok=True)

### Training Loop

def train(model, train_loader, val_loader, optimizer, scheduler, criterion, device, epochs=100):
    best_val_loss = np.inf
    epochs_no_improve = 0
    snapshots = []

    for epoch in range (1, epochs+1):
        model.train()
        running_loss = 0.0

        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()

        # snapshot ensembling
        isSnapshot = False
        if model.use_snapshots and scheduler.T_cur == 0 and epoch != 0:
            snapshots.append(model.state_dict().copy())
            torch.save(model.state_dict(), f'{save_dir}/snapshot_{len(snapshots)}.pth')
            isSnapshot = True

        # eval phase
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for inputs, targets in val_loader:
                if model.use_snapshots and len(snapshots) > 0:
                    ens_predictions = model.ensemble_predict(inputs, snapshots, device)
                    ens_predictions.append(model(inputs).squeeze())

                    # Adds current model as part of snapshot eval
                    outputs = torch.mean(torch.stack(ens_predictions), dim=0)

                    loss = criterion(outputs, targets)

                    val_loss += loss.item()
                else:
                    inputs, targets = inputs.to(device), targets.to(device)

                    outputs = model(inputs).squeeze()
                    loss = criterion(outputs, targets)

                    val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        avg_train_loss = running_loss / len(train_loader)

        # Early stopping
        isEpochImprove = False
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), f'{save_dir}/best_model.pth')
            isEpochImprove = True
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= model.early_stopping_patience:
                print(f'Stopping early with best val loss {avg_val_loss}')
                break

        print(f'Epoch {epoch}/{epochs} | Train loss: {avg_train_loss:.4f} | Val loss: {avg_val_loss:.4f} | Val RMSE: {np.sqrt(avg_val_loss):.4f}')
        print(f'\tLR: {scheduler.get_last_lr()} | T_cur: {scheduler.T_cur}')
        if isSnapshot:
            print(f'\tSnapshot taken')
        if isEpochImprove:
            print(f'\tValidation loss improved')

train(model, train_loader, val_loader, optimizer, scheduler, model.criterion, device, epochs=210)

Epoch 1/210 | Train loss: 6.5576 | Val loss: 4.4824 | Val RMSE: 2.1172
	LR: [0.009972612215893682] | T_cur: 1
	Validation loss improved
Epoch 2/210 | Train loss: 5.0056 | Val loss: 4.2806 | Val RMSE: 2.0690
	LR: [0.009890748929868663] | T_cur: 2
	Validation loss improved
Epoch 3/210 | Train loss: 4.6710 | Val loss: 4.1994 | Val RMSE: 2.0493
	LR: [0.00975530705321762] | T_cur: 3
	Validation loss improved
Epoch 4/210 | Train loss: 4.5246 | Val loss: 4.2509 | Val RMSE: 2.0618
	LR: [0.009567770515484183] | T_cur: 4
Epoch 5/210 | Train loss: 4.4213 | Val loss: 4.1208 | Val RMSE: 2.0300
	LR: [0.009330194006220302] | T_cur: 5
	Validation loss improved
Epoch 6/210 | Train loss: 4.3769 | Val loss: 4.1370 | Val RMSE: 2.0340
	LR: [0.00904518046337755] | T_cur: 6
Epoch 7/210 | Train loss: 4.3361 | Val loss: 4.0759 | Val RMSE: 2.0189
	LR: [0.008715852554974233] | T_cur: 7
	Validation loss improved
Epoch 8/210 | Train loss: 4.3176 | Val loss: 4.1511 | Val RMSE: 2.0374
	LR: [0.008345818466491111] | T

To load a snapshotted model for inference, just initialize MLP Regressor classes for all snapshots, and load a snapshot state dict into each class.

In [177]:
### Inference using saved models

model_list = []

for model_path in os.listdir(save_dir):
    model = MLPRegressor(
        input_dim=input_dim, 
        hidden_dims=hidden_dims,
        learning_rate=1e-2,
        use_snapshots=True,
        cycle_length=30,
        cycle_mult=1,
        batch_normalization=True,
        weight_decay=0.01,
        dropout_rate=0.4,
        early_stopping_patience=150,
        activation_fn=nn.ReLU()
    )

    model.load_state_dict(torch.load(f'{save_dir}/{model_path}'))
    model.to(device)
    model_list.append(model)

In [178]:
for i, model in enumerate(model_list):
    with torch.no_grad():
        test_model = model_list[i]
        test_model.eval()

        y_pred_list = []
        y_true_list = []

        for inputs, targets in val_loader:
            y_pred_batch = test_model(inputs).squeeze().cpu().numpy()
            y_true_batch = targets.cpu().numpy()
            y_pred_list.extend(y_pred_batch)
            y_true_list.extend(y_true_batch)

        y_pred = np.array(y_pred_list)
        y_true = np.array(y_true_list)

    from sklearn.metrics import mean_squared_error

    print(f"Model {i}: {np.sqrt(mean_squared_error(y_true, y_pred))}")

Model 0: 1.8869935274124146
Model 1: 1.8865877389907837
Model 2: 1.8908019065856934
Model 3: 1.8874437808990479
Model 4: 1.8957056999206543
Model 5: 1.8883732557296753
Model 6: 1.911879301071167
Model 7: 1.8845239877700806


In [179]:
with torch.no_grad():
    preds = []
    for i, model in enumerate(model_list):
        test_model = model
        test_model.eval()

        y_pred_list = []
        y_true_list = []

        for inputs, targets in val_loader:
            y_pred_batch = test_model(inputs).squeeze().cpu().numpy()
            y_true_batch = targets.cpu().numpy()
            y_pred_list.extend(y_pred_batch)
            y_true_list.extend(y_true_batch)
        
        y_pred = np.array(y_pred_list)
        y_true = np.array(y_true_list)

        preds.append(y_pred)

    y_pred_val = np.mean(preds, axis=0)

print(f"Combined mean prediction: {np.sqrt(mean_squared_error(y_true, y_pred_val))}")

Combined mean prediction: 1.8821855783462524


In [181]:
X_test_np = X_test.to_numpy().astype(np.float32)

X_test_tensor = torch.from_numpy(X_test_np).to(device)


with torch.no_grad():
    preds = []
    for i, model in enumerate(model_list):
        test_model = model
        test_model.eval()

        y_pred = test_model(X_test_tensor).squeeze().cpu().numpy()
        
        preds.extend(y_pred)
    
    preds = np.array(preds)
    y_pred = np.mean(preds, axis=0)


df_subm = pd.read_csv(
    "brist1d/sample_submission.csv",
    index_col='id',
)

df_subm['bg+1:00'] = y_pred
df_subm.to_csv('submission_mlp_regressor.csv')